In [5]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from useful_tools import tipo_de_datos

Para el EDA, necesitamos responder cuales de las columnas de esta base de datos nos ayuda a lograr los objetivos del proyecto:

- Crear un sistema de recomendacion de videojuegos para usuarios con las consultas requeridas


Y cuales relacionan esta base con las otras bases. Esos seran nuestros objetivos en este EDA. Para este proposito tambien utilizaremos el nuevo Diccionario de Datos.

In [6]:
# Cargar el conjunto de datos user_reviews
steam_games = pd.read_parquet('steam_games.parquet')

In [7]:
# Mostrar las primeras filas del DataFrame
print("Primeras y ultimas filas de steam_games:")
steam_games

Primeras y ultimas filas de steam_games:


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [8]:
# Obtener información sobre el DataFrame
print("\nInformación de steam_games:")
tipo_de_datos.datatype_per_column(steam_games)


Información de steam_games:
Columna 'publisher': [<class 'str'> <class 'NoneType'>]
Columna 'genres': [<class 'numpy.ndarray'> <class 'NoneType'>]
Columna 'app_name': [<class 'str'> <class 'NoneType'>]
Columna 'title': [<class 'str'> <class 'NoneType'>]
Columna 'url': [<class 'str'>]
Columna 'release_date': [<class 'str'> <class 'NoneType'>]
Columna 'tags': [<class 'numpy.ndarray'> <class 'NoneType'>]
Columna 'reviews_url': [<class 'str'> <class 'NoneType'>]
Columna 'specs': [<class 'numpy.ndarray'> <class 'NoneType'>]
Columna 'price': [<class 'float'>]
Columna 'early_access': [<class 'float'>]
Columna 'id': [<class 'float'>]
Columna 'developer': [<class 'str'> <class 'NoneType'>]


In [9]:
# Verificar la forma (número de filas y columnas) del DataFrame
print("\nForma de steam_games:", steam_games.shape)


Forma de steam_games: (32135, 13)


In [10]:
# Verificar valores faltantes en el DataFrame
print("\nValores faltantes en user_reviews:")
print(steam_games.isnull().sum())


Valores faltantes en user_reviews:
publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           2305
early_access       0
id                 2
developer       3299
dtype: int64


Existen varias columnas con datos nulos. Veremos como podemos solucionar esto. Primero, vemos que los datos en app_name y title son parecidos entre sí. Ya que app_name solo tiene dos datos nulos, intentaremos solucionar esos casos y mantendremos esa columna.

In [11]:
# Encontrar las filas con valores nulos en 'app_name'
null_app_name = steam_games[steam_games['app_name'].isnull()]

# Mostrar las filas con valores nulos en 'app_name'
print("Filas con valores nulos en 'app_name':")
null_app_name


Filas con valores nulos en 'app_name':


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,0.0,317160.0,None


No se puede resolver ninguno de los dos casos, por lo tanto, eliminaremos las filas.

In [12]:
# Eliminar las filas con valores nulos en 'app_name'
steam_games = steam_games.dropna(subset=['app_name'])

steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


Las columnas genre y tags poseen datos parecidos. Para completar la columna genre, generaremos una matriz de datos unicos en genre, es decir, todos los generos existentes. luego verificaremos si alguno de esos generos estan presentes en la columna tags de las lineas con datos nulos en genre. Utilizaremos los datos de tags para completar genre.

In [13]:
# Obtener todas las listas de géneros y unirlas en una sola lista
all_genres = []
for genres_list in steam_games['genres']:
    # Verificar si la lista de géneros no es nula
    if genres_list is not None:
        # Verificar si la lista de géneros no está vacía
        if len(genres_list) > 0:
            # Extender la lista de todos los géneros con los géneros de la fila actual
            all_genres.extend(genres_list)

# Eliminar duplicados y valores nulos de la lista resultante
unique_genres = list(set(all_genres))

# Mostrar la lista maestra de géneros
print("Lista maestra de géneros:")
unique_genres


Lista maestra de géneros:


['RPG',
 'Photo Editing',
 'Animation &amp; Modeling',
 'Education',
 'Sports',
 'Early Access',
 'Action',
 'Accounting',
 'Simulation',
 'Free to Play',
 'Audio Production',
 'Massively Multiplayer',
 'Adventure',
 'Indie',
 'Video Production',
 'Strategy',
 'Web Publishing',
 'Utilities',
 'Racing',
 'Casual',
 'Design &amp; Illustration',
 'Software Training']

Vamos a corregir 'Animation &amp; Modeling' y 'Design &amp; Illustration'

In [14]:
# Reemplazar las instancias en la lista maestra
unique_genres = [genre.replace('Animation &amp; Modeling', 'Animation & Modeling') for genre in unique_genres]
unique_genres = [genre.replace('Design &amp; Illustration', 'Design & Illustration') for genre in unique_genres]
unique_genres

['RPG',
 'Photo Editing',
 'Animation & Modeling',
 'Education',
 'Sports',
 'Early Access',
 'Action',
 'Accounting',
 'Simulation',
 'Free to Play',
 'Audio Production',
 'Massively Multiplayer',
 'Adventure',
 'Indie',
 'Video Production',
 'Strategy',
 'Web Publishing',
 'Utilities',
 'Racing',
 'Casual',
 'Design & Illustration',
 'Software Training']

Con esta lista de todos los generos, verificaremos, en los casos en que la columna genres tiene datos nulos o faltantes, si alguno de los generos en la lista maestra se encuentra en la lista de la columna tags y en la columna specs. Si uno o mas generos estan alli, se creará una lista con esos generos y se guardará en la columna genres.

In [15]:
# Define la función update_genres modificada
def update_genres(row):
    if (not isinstance(row['genres'], np.ndarray)) and (isinstance(row['tags'], np.ndarray)):
        matched_genres = np.array([tag for tag in row['tags'] if tag in unique_genres])
        if len(matched_genres) > 0:
            return matched_genres
        else:
            return row['genres']
    else:
        return row['genres']

# Aplica la función update_genres a cada fila del DataFrame steam_games
steam_games['genres'] = steam_games.apply(update_genres, axis=1)

/var/folders/cc/k93262dx3yz2w0blkmvzstwm0000gq/T/ipykernel_704/3194013097.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['genres'] = steam_games.apply(update_genres, axis=1)


In [16]:
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,"[Action, Indie, Casual, Sports]",Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [17]:
# Verificar valores faltantes en el DataFrame
print("\nValores faltantes en user_reviews:")
print(steam_games.isnull().sum())


Valores faltantes en user_reviews:
publisher       8050
genres           925
app_name           0
title           2048
url                0
release_date    2066
tags             162
reviews_url        1
specs            669
price           2304
early_access       0
id                 1
developer       3297
dtype: int64


In [18]:
# Define la función update_genres modificada
def update_genres2(row):
    if (not isinstance(row['genres'], np.ndarray)) and (isinstance(row['specs'], np.ndarray)):
        matched_genres = np.array([tag for tag in row['specs'] if tag in unique_genres])
        if len(matched_genres) > 0:
            return matched_genres
        else:
            return row['genres']
    else:
        return row['genres']

# Aplica la función update_genres a cada fila del DataFrame steam_games
steam_games['genres'] = steam_games.apply(update_genres2, axis=1)



/var/folders/cc/k93262dx3yz2w0blkmvzstwm0000gq/T/ipykernel_704/4272133929.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['genres'] = steam_games.apply(update_genres2, axis=1)


In [25]:
# Verificar valores faltantes en el DataFrame
print("\nValores faltantes en user_reviews:")
print(steam_games.isnull().sum())


Valores faltantes en user_reviews:
publisher       8050
genres           925
app_name           0
title           2048
url                0
release_date    2066
tags             162
reviews_url        1
specs            669
price           2304
early_access       0
id                 1
developer       3297
dtype: int64
